# Llama Indexing for Chroma

In [13]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

from IPython.display import Markdown, display


In [8]:

# Create chroma client
# chroma_client = chromadb.Client()
# chroma_client = chromadb.Client(Settings(chroma_api_impl="rest",
#                                 chroma_server_host="18.233.156.143",  # EC2 instance public IPv4
#                                 chroma_server_http_port=8000))

db = chromadb.Client(
    Settings(chroma_db_impl="duckdb+parquet", persist_directory="./chroma_db")
)

# returns a nanosecond heartbeat. Useful for making sure the client remains connected.
print("Nanosecond heartbeat on server", chroma_client.heartbeat())

# Check Existing connections
print(chroma_client.list_collections())

default_ef = embedding_functions.DefaultEmbeddingFunction()
collection = db.get_or_create_collection("fulltext")

Nanosecond heartbeat on server 1688939199295354373000
[Collection(name=fulltext)]


In [ ]:
collection.add(*doc)

In [9]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [15]:
# define embedding function
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [66]:
import pandas as pd
doc = pd.read_csv("/home/ubuntu/work/data/fulltext_docs_csvs_cleaned/fulltext_doc_1.csv")

In [67]:
doc

,0
0,{'documents': ['the implication of metabolical...
1,"{'documents': ['HAPLOMETRA CYLINDRACEA (ZEDER,..."
2,{'documents': ['Harnessing Gene Expression Net...
3,{'documents': ['Phasic Contractions in Urinary...
4,"{'documents': [""Altered Mental Status as a Nov..."
...,...
382,{'documents': ['Utility of the Physical Examin...
383,{'documents': ['Department of Internal Medicin...
384,{'documents': ['Liverpool and Macarthur Cancer...
385,{'documents': ['High Prevalence and Partner Co...


In [68]:
# turn string dictionaries into dictionary objects
import ast
ast.literal_eval(doc.iloc[0, 0])

{'documents': ['the implication of metabolically active Vibrio spp. in the digestive tract of Litopenaeus vannamei for its post-larval development\n0123456789\n\nEstefanía Garibay-Valdez \nCentro de Investigación en Alimentos y Desarrollo A.C (CIAD)\nCarretera a La Victoria S/NCP. 83304Hermosillo, SonoraMexico\n\nLuis Rafael Martínez-Córdova \nDepartamento de Investigaciones Científicas y Tecnológicas (DICTUS)\nUniversidad de Sonora\nBlvd. Luis Donaldo Colosio S/N\nCP. 83000Hermosillo, SonoraMexico\n\nMarco A López-Torres \nDepartamento de Investigaciones Científicas y Tecnológicas (DICTUS)\nUniversidad de Sonora\nBlvd. Luis Donaldo Colosio S/N\nCP. 83000Hermosillo, SonoraMexico\n\nF Javier Almendariz-Tapia \nDepartamento de Ingeniería Química y Metalurgia\nUniversidad de Sonora\nBlvd. Luis Donaldo Colosio S/N\nCP. 83000Hermosillo, SonoraMexico\n\nMarcel Martínez-Porchas \nCentro de Investigación en Alimentos y Desarrollo A.C (CIAD)\nCarretera a La Victoria S/NCP. 83304Hermosillo, Sono

In [52]:
import langchain

In [72]:
test = ast.literal_eval(doc.iloc[0, 0])['documents']

In [18]:
documents = SimpleDirectoryReader('/home/ubuntu/work/data/fulltext_docs_csvs/', num_files_limit= 5).load_data()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 documents = SimpleDirectoryReader('/home/ubuntu/work/data/fulltext_docs_csvs/').load_dat     │
│   2                                                                                              │
│   3 # set up ChromaVectorStore and load in data                                                  │
│   4 vector_store = ChromaVectorStore(chroma_collection=collection)                               │
│                                                                                                  │
│ /home/ubuntu/anaconda3/envs/chroma/lib/python3.10/site-packages/llama_index/readers/file/base.py │
│ :195 in load_data                                                                                │
│                                                                                                  │
│   192 │   │   │   │   │   reader_cls = DEFAULT_FILE_READER_CLS[file_suffix]                      │
│   193 │   │   │   │   │   self.file_extractor[file_suffix] = reader_cls()                        │
│   194 │   │   │   │   reader = self.file_extractor[file_suffix]                                  │
│ ❱ 195 │   │   │   │   docs = reader.load_data(input_file, metadata=metadata or {})               │
│   196 │   │   │   │                                                                              │
│   197 │   │   │   │   # iterate over docs if needed                                              │
│   198 │   │   │   │   if self.filename_as_id:                                                    │
│                                                                                                  │
│ /home/ubuntu/anaconda3/envs/chroma/lib/python3.10/site-packages/llama_index/readers/file/tabular │
│ _reader.py:96 in load_data                                                                       │
│                                                                                                  │
│    93 │                                                                                          │
│    94 │   def load_data(self, file: Path, metadata: Optional[Dict] = None) -> List[Document]:    │
│    95 │   │   """Parse file."""                                                                  │
│ ❱  96 │   │   df = pd.read_csv(file, **self._pandas_config)                                      │
│    97 │   │                                                                                      │
│    98 │   │   text_list = df.apply(                                                              │
│    99 │   │   │   lambda row: (self._col_joiner).join(row.astype(str).tolist()), axis=1          │
│                                                                                                  │
│ /home/ubuntu/anaconda3/envs/chroma/lib/python3.10/site-packages/pandas/util/_decorators.py:211   │
│ in wrapper                                                                                       │
│                                                                                                  │
│   208 │   │   │   │   │   raise TypeError(msg)                                                   │
│   209 │   │   │   │   else:                                                                      │
│   210 │   │   │   │   │   kwargs[new_arg_name] = new_arg_value                                   │
│ ❱ 211 │   │   │   return func(*args, **kwargs)                                                   │
│   212 │   │                                                                                      │
│   213 │   │   return cast(F, wrapper)                                                            │
│   214                                                      

In [ ]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=collection)


In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [ ]:

# Query Data
query_engine = index.as_query_engine(chroma_collection=chroma_collection)
response = query_engine.query("What did the author do growing up?")
display(Markdown(f"<b>{response}</b>"))

Check Work

In [ ]:
collection.count()